# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacitypr2 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacitypr2')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1:

In [8]:
# Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
#First: Create session_library table
try:
    query= "Create table if not exists session_library"
    query=query+ "(artist_name text, song_title text,song_length float, sessionid text,\
    itemInsession text, primary key(sessionid,itemInsession))"

    session.execute(query)       
except Exception as e:
    print(e)

In [9]:
# second: read from csv file and insert into session library table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "Insert into session_library (artist_name , song_title ,song_length , sessionid ,itemInsession)"
        query = query + "values(%s,%s,%s,%s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[0], line[9],float(line[5]),line[8],line[3]))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
##  select query to get data
try:
    query="select artist_name,song_title,song_length from session_library where sessionid='338' and itemInsession='4'"
    rows=session.execute(query)
    
except e:
    print(e)

for row in rows:
        print(row.artist_name,",",row.song_title,",",row.song_length)

Faithless , Music Matters (Mark Knight Dub) , 495.30731201171875


I model the session library table by setting a composite primary key by sessionid and itemInsession (partition keys)
Then after running our query we got one row where sessionid='338' and itemInsession='4' 


## Query2

In [11]:
##  Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
#First Create users library table
try:
    query= "Create table if not exists users_library"
    query=query+ "(artist_name text, song_title text,first_name text,last_name text, sessionid text,\
    userid text, itemInsession text, primary key((userid,sessionid),itemInsession))"

    session.execute(query)       
except Exception as e:
    print(e)

                    

In [12]:
#second insert data to users library table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "Insert into users_library (artist_name , song_title ,first_name ,last_name , sessionid ,\
    userid, itemInsession)"
        query = query + "values(%s,%s,%s,%s,%s,%s,%s)"
        ##  Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[0], line[9],line[1],line[4],line[8],line[10],line[3]))

In [13]:
## select name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
try:
    query="select artist_name,song_title,first_name,last_name from users_library where userid='10' and sessionid='182'"
    rows=session.execute(query)
    
except e:
    print(e)

for row in rows:
        print(row.artist_name,",",row.song_title,",",row.first_name,",",row.last_name)

                  

Down To The Bone , Keep On Keepin' On , Sylvie , Cruz
Three Drives , Greece 2000 , Sylvie , Cruz
Sebastien Tellier , Kilometer , Sylvie , Cruz
Lonnie Gordon , Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , Sylvie , Cruz


I model users library table by setting partition key = userid+sessionid and clustering column is itemInsession , after running query userid=10 and sessionid=182 we got songs played in specific session by specific user sorted by iteminsession

## Query3

In [14]:
##  Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
#First create songs_library table
try:
    query= "Create table if not exists songs_library"
    query=query+ "(song_title text,first_name text,last_name text,userid text,  primary key(song_title,userid))"

    session.execute(query)       
except Exception as e:
    print(e) 


In [15]:
#second: insert data into songs library table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "Insert into songs_library (song_title ,first_name ,last_name  ,userid )"
        query = query + "values(%s,%s,%s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], line[1],line[4],line[10]))

In [16]:
#select every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
#First create songs_library
try:
    query="select first_name,last_name from songs_library where song_title='All Hands Against His Own'"
    rows=session.execute(query)
    
except e:
    print(e)

for row in rows:
        print(row.first_name,",",row.last_name)

                  

Jacqueline , Lynch
Tegan , Levine
Sara , Johnson


I model songs library table by setting composite primary key song title+user id to be able to query user name who listened to specific song 

### Drop the tables before closing out the sessions

In [17]:
## Drop the table before closing out the sessions

try:
    #Drop session library table
    query="DROP TABLE if exists udacitypr2.session_library"
    session.execute(query)
    #Drop users library table
    query2="DROP TABLE if exists udacitypr2.users_library"
    session.execute(query2)
    #Drop songs library table
    query3="DROP TABLE if exists udacitypr2.songs_library"
    session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()